## Prequisites

### Create new sim app

In [1]:
# cp ./src/lib/level_manager.py ./build/simapp/bundle/usr/local/lib/python3.5/dist-packages/rl_coach/level_manager.py

In [2]:
#Only when you have changes to SimApp
# !python3 sim_app_bundler.py --tar

### Customize environment, reward, preset

In [3]:
#experiment_name = 'lidar-simple-5CNN-2Dense-Narrow_3Cars'
#experiment_name = 'lidar-advanced-3CNN-1Dense-Wide_3Cars'
#experiment_name = 'lidar-advanced-3CNN-2Dense-Narrow_3Cars'
#experiment_name = 'lidar-advanced-5CNN-1Dense-Wide_3Cars'
experiment_name = 'lidar-advanced-5CNN-2Dense-Narrow_5Cars_run4'


#reward
if 'advanced' in experiment_name:
    !cp customize/reward_advanced.py src/markov/rewards/reward_function.py
if 'simple' in experiment_name:
    !cp customize/reward_simple.py src/markov/rewards/reward_function.py

# preset
if '5CNN' in experiment_name and 'Narrow' in experiment_name:
    !cp customize/preset_5CNN_2Dense_narrow.py src/markov/presets/preset.py
if '5CNN' in experiment_name and 'Wide' in experiment_name:
    !cp customize/preset_5CNN_1Dense_wide.py src/markov/presets/preset.py

if '3CNN' in experiment_name and 'Narrow' in experiment_name:
    !cp customize/preset_3CNN_2Dense_narrow.py src/markov/presets/preset.py
if '3CNN' in experiment_name and 'Wide' in experiment_name:
    !cp customize/preset_3CNN_1Dense_wide.py src/markov/presets/preset.py

# the environment file
if '5Cars' in experiment_name:
    !cp customize/deepracer_racetrack_env_lidar_5cars.py src/markov/environments/deepracer_racetrack_env.py
if '3Cars' in experiment_name:
    !cp customize/deepracer_racetrack_env_lidar_3cars.py src/markov/environments/deepracer_racetrack_env.py
    
#!cp customize/deepracer_racetrack_env_shallow_advanced-lidar-turns.py src/markov/environments/deepracer_racetrack_env.py



### Imports

In [4]:
import boto3
import sagemaker
import sys
import os
import re
import numpy as np
import subprocess
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
from time import gmtime, strftime
import time
from IPython.display import Markdown
from markdown_helper import *

### Initializing basic parameters

In [5]:
# Select the instance type
instance_type = "ml.p3.16xlarge"
#instance_type = "ml.p2.xlarge"
# instance_type = "ml.c5.4xlarge"

# Starting SageMaker session
sage_session = sagemaker.session.Session()

# stick to 1,2,4,8
# num_simulation_workers = 1

# Create unique job name.
job_name_prefix = 'sahika-lidar-oldcar-tf12-wide-ic'

# Duration of job in seconds (1 hours)
job_duration_in_seconds = 60 * 60 * 20

# AWS Region
aws_region = sage_session.boto_region_name
if aws_region not in ["us-west-2", "us-east-1", "eu-west-1"]:
    raise Exception("This notebook uses RoboMaker which is available only in US East (N. Virginia),"
                    "US West (Oregon) and EU (Ireland). Please switch to one of these regions.")

### Setup S3 bucket
Set up the linkage and authentication to the S3 bucket that we want to use for checkpoint and metadata.

In [6]:
# S3 bucket
s3_bucket = job_name_prefix #"sahika-multicar-stereo-camera"
#s3_bucket = "sahika-multicar-lidar-small-angle"

# SDK appends the job name and output folder
s3_output_path = 's3://{}/'.format(s3_bucket)

#Ensure that the S3 prefix contains the keyword 'sagemaker'
s3_prefix = job_name_prefix + "-sagemaker-" + strftime("%y%m%d-%H%M%S", gmtime())

# Get the AWS account id of this account
sts = boto3.client("sts")
account_id = sts.get_caller_identity()['Account']

print("Using s3 bucket {}".format(s3_bucket))
print("Model checkpoints and other metadata will be stored at: \ns3://{}/{}".format(s3_bucket, s3_prefix))

Using s3 bucket sahika-lidar-oldcar-tf12-wide-ic
Model checkpoints and other metadata will be stored at: 
s3://sahika-lidar-oldcar-tf12-wide-ic/sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456


### Create an IAM role
Either get the execution role when running from a SageMaker notebook `role = sagemaker.get_execution_role()` or, when running from local machine, use utils method `role = get_execution_role('role_name')` to create an execution role.

In [7]:
try:
    sagemaker_role = sagemaker.get_execution_role()
except:
    sagemaker_role = get_execution_role('sagemaker')

print("Using Sagemaker IAM role arn: \n{}".format(sagemaker_role))

Using Sagemaker IAM role arn: 
arn:aws:iam::828753602420:role/service-role/AmazonSageMaker-ExecutionRole-20190604T142184


> Please note that this notebook cannot be run in `SageMaker local mode` as the simulator is based on AWS RoboMaker service.

### Permission setup for invoking AWS RoboMaker from this notebook
In order to enable this notebook to be able to execute AWS RoboMaker jobs, we need to add one trust relationship to the default execution role of this notebook.

In [8]:
# display(Markdown(generate_help_for_robomaker_trust_relationship(sagemaker_role)))

### Permission setup for Sagemaker to S3 bucket

The sagemaker writes the Redis IP address, models to the S3 bucket. This requires PutObject permission on the bucket. Make sure the sagemaker role you are using as this permissions.

In [9]:
# display(Markdown(generate_s3_write_permission_for_sagemaker_role(sagemaker_role)))

### Build and push docker image

The file ./Dockerfile contains all the packages that are installed into the docker. Instead of using the default sagemaker container. We will be using this docker container.

In [10]:
%%time
from copy_to_sagemaker_container import get_sagemaker_docker, copy_to_sagemaker_container, get_custom_image_name
cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
# repo name
repository_short_name = job_name_prefix + "-%s" % cpu_or_gpu
custom_image_name = get_custom_image_name(repository_short_name)

try:
    print("Copying files from your notebook to existing sagemaker container")
    sagemaker_docker_id = get_sagemaker_docker(repository_short_name)
    copy_to_sagemaker_container(sagemaker_docker_id, repository_short_name)
except Exception as e:
    print("Creating sagemaker container")
    docker_build_args = {
        'CPU_OR_GPU': cpu_or_gpu, 
        'AWS_REGION': boto3.Session().region_name,
    }
    custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
    print("Using ECR image %s" % custom_image_name)
    
# print("Creating sagemaker container")
# docker_build_args = {
#     'CPU_OR_GPU': cpu_or_gpu, 
#     'AWS_REGION': boto3.Session().region_name,
# }
# custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
# print("Using ECR image %s" % custom_image_name)


Copying files from your notebook to existing sagemaker container
docker images sahika-lidar-oldcar-tf12-wide-ic-gpu | sed -n 2,2p
Sagemaker docker id : 7acfdc6f0522
docker run -d -t 7acfdc6f0522
docker exec -d 88c4762bce9659b776568fc282c11b42551f655c1420e2e28c858216f2619a5f rm -rf /opt/amazon/markov
docker cp ./src/markov 88c4762bce9659b776568fc282c11b42551f655c1420e2e28c858216f2619a5f:/opt/amazon/markov
============ Copied Markov scripts to sagemaker docker ============ 
 
docker ps -l|sed -n 2,2p
docker commit 88c4762bce96 sahika-lidar-oldcar-tf12-wide-ic-gpu
============ Commited all the changes to docker ============ 
 
CPU times: user 29.5 ms, sys: 27.9 ms, total: 57.4 ms
Wall time: 1.41 s


In [11]:
print("Using ECR image %s" % custom_image_name)

Using ECR image 828753602420.dkr.ecr.us-west-2.amazonaws.com/sahika-lidar-oldcar-tf12-wide-ic-gpu


### Configure VPC

Since SageMaker and RoboMaker have to communicate with each other over the network, both of these services need to run in VPC mode. This can be done by supplying subnets and security groups to the job launching scripts.  
We will check if the deepracer-vpc stack is created and use it if present (This is present if the AWS Deepracer console is used atleast once to create a model). Else we will use the default VPC stack.

In [12]:
ec2 = boto3.client('ec2')

#
# Check if the user has Deepracer-VPC and use that if its present. This will have all permission.
# This VPC will be created when you have used the Deepracer console and created one model atleast
# If this is not present. Use the default VPC connnection
#
deepracer_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups']\
                             if group['GroupName'].startswith("deepracer-vpc")]
if(deepracer_security_groups):
    print("Using the DeepRacer VPC stacks")
    deepracer_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] \
                     if "Tags" in vpc for val in vpc['Tags'] \
                     if val['Value'] == 'deepracer-vpc'][0]
    deepracer_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                         if subnet["VpcId"] == deepracer_vpc]
else:
    print("Using the default VPC stacks")
    deepracer_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] if vpc["IsDefault"] == True][0]

    deepracer_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups'] \
                                 if 'VpcId' in group and group["GroupName"] == "default" and group["VpcId"] == deepracer_vpc]

    deepracer_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                         if subnet["VpcId"] == deepracer_vpc and subnet['DefaultForAz']==True]

print("Using VPC:", deepracer_vpc)
print("Using security group:", deepracer_security_groups)
print("Using subnets:", deepracer_subnets)

Using the default VPC stacks
Using VPC: vpc-b418facc
Using security group: ['sg-fca8088c']
Using subnets: ['subnet-57a5de2e', 'subnet-67cc983d', 'subnet-b63155fd']


### Create Route Table
A SageMaker job running in VPC mode cannot access S3 resourcs. So, we need to create a VPC S3 endpoint to allow S3 access from SageMaker container. To learn more about the VPC mode, please visit [this link.](https://docs.aws.amazon.com/sagemaker/latest/dg/train-vpc.html)

In [13]:
#TODO: Explain to customer what CREATE_ROUTE_TABLE is doing
CREATE_ROUTE_TABLE = True

def create_vpc_endpoint_table():
    print("Creating ")
    try:
        route_tables = [route_table["RouteTableId"] for route_table in ec2.describe_route_tables()['RouteTables']\
                        if route_table['VpcId'] == deepracer_vpc]
    except Exception as e:
        if "UnauthorizedOperation" in str(e):
            display(Markdown(generate_help_for_s3_endpoint_permissions(sagemaker_role)))
        else:
            display(Markdown(create_s3_endpoint_manually(aws_region, deepracer_vpc)))
        raise e

    print("Trying to attach S3 endpoints to the following route tables:", route_tables)
    
    if not route_tables:
        raise Exception(("No route tables were found. Please follow the VPC S3 endpoint creation "
                         "guide by clicking the above link."))
    try:
        ec2.create_vpc_endpoint(DryRun=False,
                                VpcEndpointType="Gateway",
                                VpcId=deepracer_vpc,
                                ServiceName="com.amazonaws.{}.s3".format(aws_region),
                                RouteTableIds=route_tables)
        print("S3 endpoint created successfully!")
    except Exception as e:
        if "RouteAlreadyExists" in str(e):
            print("S3 endpoint already exists.")
        elif "UnauthorizedOperation" in str(e):
            display(Markdown(generate_help_for_s3_endpoint_permissions(role)))
            raise e
        else:
            display(Markdown(create_s3_endpoint_manually(aws_region, default_vpc)))
            raise e

if CREATE_ROUTE_TABLE:
    create_vpc_endpoint_table()

Creating 
Trying to attach S3 endpoints to the following route tables: ['rtb-0d549376']
S3 endpoint already exists.


## Setup the environment

The environment is defined in a Python file called “deepracer_racetrack_env.py” and the file can be found at `src/markov/environments/`. This file implements the gym interface for our Gazebo based RoboMakersimulator. This is a common environment file used by both SageMaker and RoboMaker. The environment variable - `NODE_TYPE` defines which node the code is running on. So, the expressions that have `rospy` dependencies are executed on RoboMaker only.  

We can experiment with different reward functions by modifying `reward_function` in `src/markov/rewards/`. Action space and steering angles can be changed by modifying `src/markov/actions/`.json file

### Configure the preset for RL algorithm

The parameters that configure the RL training job are defined in `src/markov/presets/`. Using the preset file, you can define agent parameters to select the specific agent algorithm. We suggest using Clipped PPO for this example.  
You can edit this file to modify algorithm parameters like learning_rate, neural network structure, batch_size, discount factor etc.

In [14]:
# Uncomment the pygmentize code lines to see the code

# Environmental File
#!pygmentize src/markov/environments/deepracer_racetrack_env.py

# Reward function
#!pygmentize src/markov/rewards/default.py

# Action space
#!pygmentize src/markov/actions/model_metadata_10_state.json

# Preset File
#!pygmentize src/markov/presets/default.py
#!pygmentize src/markov/presets/preset_attention_layer.py

### Copy custom files to S3 bucket so that sagemaker & robomaker can pick it up

In [15]:
s3_location = "s3://%s/%s" % (s3_bucket, s3_prefix)
print(s3_location)

# Clean up the previously uploaded files
!aws s3 rm --recursive {s3_location}


s3://sahika-lidar-oldcar-tf12-wide-ic/sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456


### Copy the environment file to the s3 bucket


In [16]:
# recursive copy
!aws s3 cp src/markov {s3_location}/markov/ --recursive

# Make any changes to the environment and preset files below and upload these files
!aws s3 cp src/markov/environments/deepracer_racetrack_env.py {s3_location}/environments/deepracer_racetrack_env.py


upload: src/markov/.ipynb_checkpoints/s3_client-checkpoint.py to s3://sahika-lidar-oldcar-tf12-wide-ic/sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456/markov/.ipynb_checkpoints/s3_client-checkpoint.py
upload: src/markov/.ipynb_checkpoints/evaluation_worker-checkpoint.py to s3://sahika-lidar-oldcar-tf12-wide-ic/sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456/markov/.ipynb_checkpoints/evaluation_worker-checkpoint.py
upload: src/markov/__init__.py to s3://sahika-lidar-oldcar-tf12-wide-ic/sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456/markov/__init__.py
upload: src/markov/defaults.py to s3://sahika-lidar-oldcar-tf12-wide-ic/sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456/markov/defaults.py
upload: src/markov/environments/__init__.py to s3://sahika-lidar-oldcar-tf12-wide-ic/sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456/markov/environments/__init__.py
upload: src/markov/environments/.ipynb_checkpoints/dump2s3_deepracer_racetrack_env_advanced_sa

### change the reward function

In [17]:
#!aws s3 cp src/markov/rewards/default.py {s3_location}/rewards/reward_function.py
#!aws s3 cp src/markov/rewards/wide_throttlepenalty.py {s3_location}/rewards/reward_function.py
#!aws s3 cp src/markov/rewards/wedge_reward.py {s3_location}/rewards/reward_function.py
#!aws s3 cp src/markov/rewards/advanced_change_lane.py {s3_location}/rewards/reward_function.py
#!aws s3 cp src/markov/rewards/basic_lane_only.py {s3_location}/rewards/reward_function.py
#!aws s3 cp src/markov/rewards/reward_function_shallow_advanced.py {s3_location}/rewards/reward_function.py
#!aws s3 cp src/markov/rewards/simple_V2.py {s3_location}/rewards/reward_function.py

!aws s3 cp src/markov/rewards/reward_function.py {s3_location}/rewards/reward_function.py


upload: src/markov/rewards/reward_function.py to s3://sahika-lidar-oldcar-tf12-wide-ic/sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456/rewards/reward_function.py


### change the action space

In [18]:
#!aws s3 cp src/markov/actions/model_metadata_10_state-throtte_1.json {s3_location}/model_metadata.json
#!aws s3 cp src/markov/actions/model_metadata_10_state-throtte_3.json {s3_location}/model_metadata.json
# !aws s3 cp src/markov/actions/model_metadata_10_state-throtte_3-slow_1.json {s3_location}/model_metadata.json
# !aws s3 cp src/markov/actions/model_metadata_10_state-throtte_3-slow_2.json {s3_location}/model_metadata.json
# !aws s3 cp src/markov/actions/model_metadata_10_state.json {s3_location}/model_metadata.json
# !aws s3 cp src/markov/actions/model_metadata_10_state-steering_20.json {s3_location}/model_metadata.json
!aws s3 cp src/markov/actions/model_metadata.json {s3_location}/model_metadata.json


upload: src/markov/actions/model_metadata.json to s3://sahika-lidar-oldcar-tf12-wide-ic/sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456/model_metadata.json


### change the network or hyper-parameters

In [19]:
#!aws s3 cp src/markov/presets/preset_lidar_stereo_shallow.py {s3_location}/presets/preset.py
#!aws s3 cp src/markov/presets/preset_lidar_stereo_semi_shallow.py {s3_location}/presets/preset.py
#!aws s3 cp src/markov/presets/preset_lidar_stereo_attention.py {s3_location}/presets/preset.py
#!aws s3 cp src/markov/presets/preset_lidar_stereo_conv1D.py {s3_location}/presets/preset.py
#!aws s3 cp src/markov/presets/preset_lidar_stereo_deep.py {s3_location}/presets/preset.py

# !aws s3 cp src/markov/presets/preset_vgg_attention.py {s3_location}/presets/preset.py
# !aws s3 cp src/markov/presets/preset_vgg_attention_bnorm.py {s3_location}/presets/preset.py
# !aws s3 cp src/markov/presets/preset_shallow_attention.py {s3_location}/presets/preset.py

!aws s3 cp src/markov/presets/preset.py {s3_location}/presets/preset.py


upload: src/markov/presets/preset.py to s3://sahika-lidar-oldcar-tf12-wide-ic/sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456/presets/preset.py


### Train the RL model using the Python SDK Script mode

Next, we define the following algorithm metrics that we want to capture from cloudwatch logs to monitor the training progress. These are algorithm specific parameters and might change for different algorithm. We use [Clipped PPO](https://coach.nervanasys.com/algorithms/policy_optimization/cppo/index.html) for this example.

In [20]:
metric_definitions = [
    # Training> Name=main_level/agent, Worker=0, Episode=19, Total reward=-102.88, Steps=19019, Training iteration=1
    {'Name': 'reward-training',
     'Regex': '^Training>.*Total reward=(.*?),'},
    
    # Policy training> Surrogate loss=-0.32664725184440613, KL divergence=7.255815035023261e-06, Entropy=2.83156156539917, training epoch=0, learning_rate=0.00025
    {'Name': 'ppo-surrogate-loss',
     'Regex': '^Policy training>.*Surrogate loss=(.*?),'},
     {'Name': 'ppo-entropy',
     'Regex': '^Policy training>.*Entropy=(.*?),'},
   
    # Testing> Name=main_level/agent, Worker=0, Episode=19, Total reward=1359.12, Steps=20015, Training iteration=2
    {'Name': 'reward-testing',
     'Regex': '^Testing>.*Total reward=(.*?),'},
]

We use the RLEstimator for training RL jobs.

1. Specify the source directory which has the environment file, preset and training code.
2. Specify the entry point as the training code
3. Specify the choice of RL toolkit and framework. This automatically resolves to the ECR path for the RL Container.
4. Define the training parameters such as the instance count, instance type, job name, s3_bucket and s3_prefix for storing model checkpoints and metadata. **Only 1 training instance is supported for now.**
4. Set the RLCOACH_PRESET as "deepracer" for this example.
5. Define the metrics definitions that you are interested in capturing in your logs. These can also be visualized in CloudWatch and SageMaker Notebooks.

In [21]:
estimator = RLEstimator(entry_point="training_worker.py",
                        source_dir='src',
                        image_name=custom_image_name,
                        dependencies=["common/"],
                        role=sagemaker_role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        metric_definitions=metric_definitions,
                        train_max_run=job_duration_in_seconds,
                        hyperparameters={
                            "s3_bucket": s3_bucket,
                            "s3_prefix": s3_prefix,
                            "aws_region": aws_region,
                            "preset_s3_key": "%s/presets/preset.py"% s3_prefix,
                            "model_metadata_s3_key": "%s/model_metadata.json" % s3_prefix,
                            "environment_s3_key": "%s/environments/deepracer_racetrack_env.py" % s3_prefix,
#                             "pretrained_s3_bucket": "yunzhe-multicar",
#                             "pretrained_s3_prefix": "yunzhe-singlecar-oldcar-sagemaker-191007-172132"
                        },
                        subnets=deepracer_subnets,
                        security_group_ids=deepracer_security_groups,
                    )

estimator.fit(wait=False, job_name=s3_prefix)
job_name = estimator.latest_training_job.job_name
print("Training job: %s" % job_name)

Training job: sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456


### Start the Robomaker job

In [22]:
robomaker = boto3.client("robomaker")

### Create Simulation Application

In [23]:
# robomaker_s3_key = 'sim_app_distributed/gsaur_sim_app_new.tar.gz'
# robomaker_source = {'s3Bucket': "gsaur-test",
#                     's3Key': robomaker_s3_key,
#                     'architecture': "X86_64"}
robomaker_s3_key = 'robomaker/simulation_ws.tar.gz'
robomaker_source = {'s3Bucket': s3_bucket,
                    's3Key': robomaker_s3_key,
                    'architecture': "X86_64"}
simulation_software_suite={'name': 'Gazebo',
                           'version': '7'}
robot_software_suite={'name': 'ROS',
                      'version': 'Kinetic'}
rendering_engine={'name': 'OGRE',
                  'version': '1.x'}

Download the DeepRacer bundle provided by RoboMaker service and upload it in our S3 bucket to create a RoboMaker Simulation Application

In [24]:
!aws s3 cp ./build/output.tar.gz s3://{s3_bucket}/{robomaker_s3_key}

upload: build/output.tar.gz to s3://sahika-lidar-oldcar-tf12-wide-ic/robomaker/simulation_ws.tar.gz


In [25]:
app_name = "sahika-multicar-oldcar-" + strftime("%y%m%d-%H%M%S", gmtime())

print(app_name)
try:
    response = robomaker.create_simulation_application(name=app_name,
                                                       sources=[robomaker_source],
                                                       simulationSoftwareSuite=simulation_software_suite,
                                                       robotSoftwareSuite=robot_software_suite,
                                                       renderingEngine=rendering_engine)
    simulation_app_arn = response["arn"]
    print("Created a new simulation app with ARN:", simulation_app_arn)
except Exception as e:
    if "AccessDeniedException" in str(e):
        display(Markdown(generate_help_for_robomaker_all_permissions(role)))
        raise e
    else:
        raise e

sahika-multicar-oldcar-191205-064523
Created a new simulation app with ARN: arn:aws:robomaker:us-west-2:828753602420:simulation-application/sahika-multicar-oldcar-191205-064523/1575528323694


### Launch the Simulation job on RoboMaker

We create [AWS RoboMaker](https://console.aws.amazon.com/robomaker/home#welcome) Simulation Jobs that simulates the environment and shares this data with SageMaker for training. 

In [26]:
available_tracks = ["reinvent_base", # 0
                    "AWS_track", # 1
                    "Tokyo_Training_track", #2
                    "Virtual_May19_Train_track", #3 (london)
                    "reInvent2018_36inch", #4
                    "reInvent2018_mirror", #5
                    "reInvent2019_track"] #6

training_tracks_indices = [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
#training_tracks_indices = [6]
training_tracks_indices = [6, 6, 6, 6, 6, 6, 6, 6]

num_simulation_workers = len(training_tracks_indices)



envriron_vars = {
    "WORLD_NAME": "reinvent_base",
    "KINESIS_VIDEO_STREAM_NAME": "SilverstoneStream",
    "PYTHONPATHS":"wohooo this worked",
    "ALTERNATE_DRIVING_DIRECTION": "false",
    "SAGEMAKER_SHARED_S3_BUCKET": s3_bucket,
    "SAGEMAKER_SHARED_S3_PREFIX": s3_prefix,
    "TRAINING_JOB_ARN": job_name,
    "APP_REGION": aws_region,
    "METRIC_NAME": "TrainingRewardScore",
    "METRIC_NAMESPACE": "AWSDeepRacer",
    "REWARD_FILE_S3_KEY": "%s/rewards/reward_function.py" % s3_prefix,
    "MODEL_METADATA_FILE_S3_KEY": "%s/model_metadata.json" % s3_prefix,
    "METRICS_S3_BUCKET": s3_bucket,
    "METRICS_S3_OBJECT_KEY": s3_prefix + "/training_metrics.json",
    "TARGET_REWARD_SCORE": "None",
    "ROBOMAKER_SIMULATION_JOB_ACCOUNT_ID": account_id
}

simulation_application = {"application":simulation_app_arn,
                          "launchConfig": {"packageName": "deepracer_simulation_environment",
                                           "launchFile": "distributed_training.launch",
                                           "environmentVariables": envriron_vars}
                         }

vpcConfig = {"subnets": deepracer_subnets,
             "securityGroups": deepracer_security_groups,
             "assignPublicIp": True}

responses = []
for job_no in range(num_simulation_workers):
    simulation_application["launchConfig"]["environmentVariables"]["WORLD_NAME"] = available_tracks[training_tracks_indices[job_no]]
    client_request_token = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    response =  robomaker.create_simulation_job(iamRole=sagemaker_role,
                                            clientRequestToken=client_request_token,
                                            maxJobDurationInSeconds=job_duration_in_seconds,
                                            failureBehavior="Fail",
                                            simulationApplications=[simulation_application],
                                            vpcConfig=vpcConfig
                                            )
    print(response)
    responses.append(response)
 

{'ResponseMetadata': {'RequestId': '1d8768ed-7660-4f84-ae73-93037f8795c8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 05 Dec 2019 06:45:25 GMT', 'content-type': 'application/json', 'content-length': '2605', 'connection': 'keep-alive', 'x-amzn-requestid': '1d8768ed-7660-4f84-ae73-93037f8795c8', 'x-amz-apigw-id': 'EN8cnHZPvHcFpfQ=', 'x-amzn-trace-id': 'Root=1-5de8a783-ca5d27366bbaede8dbc4b67c'}, 'RetryAttempts': 0}, 'arn': 'arn:aws:robomaker:us-west-2:828753602420:simulation-job/sim-5gnqcb4wgnjy', 'status': 'Pending', 'lastUpdatedAt': datetime.datetime(2019, 12, 5, 6, 45, 25, tzinfo=tzlocal()), 'failureBehavior': 'Fail', 'clientRequestToken': '2019-12-05-06-45-23', 'loggingConfig': {'recordAllRosTopics': False}, 'maxJobDurationInSeconds': 72000, 'simulationTimeMillis': 0, 'iamRole': 'arn:aws:iam::828753602420:role/service-role/AmazonSageMaker-ExecutionRole-20190604T142184', 'simulationApplications': [{'application': 'arn:aws:robomaker:us-west-2:828753602420:simulation-applicati

In [27]:
print("Created the following jobs:")
job_arns = [response["arn"] for response in responses]
for response in responses:
    print("Job ARN", response["arn"])

Created the following jobs:
Job ARN arn:aws:robomaker:us-west-2:828753602420:simulation-job/sim-5gnqcb4wgnjy
Job ARN arn:aws:robomaker:us-west-2:828753602420:simulation-job/sim-3djt2c4yxjcj
Job ARN arn:aws:robomaker:us-west-2:828753602420:simulation-job/sim-cwck722gh4bk
Job ARN arn:aws:robomaker:us-west-2:828753602420:simulation-job/sim-4lcp9fs81k6q
Job ARN arn:aws:robomaker:us-west-2:828753602420:simulation-job/sim-wyq8rsft8vhc
Job ARN arn:aws:robomaker:us-west-2:828753602420:simulation-job/sim-ck8f52fg5y6k
Job ARN arn:aws:robomaker:us-west-2:828753602420:simulation-job/sim-fvd48kbwfqv0
Job ARN arn:aws:robomaker:us-west-2:828753602420:simulation-job/sim-2cbst0sj20bw


In [28]:

print("S3_MODEL_SAVE_PREFIX = '%s'" %experiment_name)

arn_str = list()
for response in responses:
    arn_str.append(response["arn"].split('/')[-1])
print('stream_name_list =' + str(arn_str))
print("sagemaker_simapp_name_list = ['" + str(job_name) + "']")

print("s3_bucket = '%s'" %s3_bucket)
print("s3_prefix = '%s'" %s3_prefix)
print('simulation_app_arn = "%s"' %simulation_app_arn)

#print('s3 location:', s3_location)
#print(available_tracks)

S3_MODEL_SAVE_PREFIX = 'lidar-advanced-5CNN-2Dense-Narrow_5Cars_run4'
stream_name_list =['sim-5gnqcb4wgnjy', 'sim-3djt2c4yxjcj', 'sim-cwck722gh4bk', 'sim-4lcp9fs81k6q', 'sim-wyq8rsft8vhc', 'sim-ck8f52fg5y6k', 'sim-fvd48kbwfqv0', 'sim-2cbst0sj20bw']
sagemaker_simapp_name_list = ['sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456']
s3_bucket = 'sahika-lidar-oldcar-tf12-wide-ic'
s3_prefix = 'sahika-lidar-oldcar-tf12-wide-ic-sagemaker-191205-064456'
simulation_app_arn = "arn:aws:robomaker:us-west-2:828753602420:simulation-application/sahika-multicar-oldcar-191205-064523/1575528323694"


### Visualizing the simulations in RoboMaker
You can visit the RoboMaker console to visualize the simulations or run the following cell to generate the hyperlinks.

In [29]:
display(Markdown(generate_robomaker_links(job_arns, aws_region)))

> Click on the following links for visualization of simulation jobs on RoboMaker Console
- [Simulation 1](https://us-west-2.console.aws.amazon.com/robomaker/home?region=us-west-2#simulationJobs/sim-5gnqcb4wgnjy)  
- [Simulation 2](https://us-west-2.console.aws.amazon.com/robomaker/home?region=us-west-2#simulationJobs/sim-3djt2c4yxjcj)  
- [Simulation 3](https://us-west-2.console.aws.amazon.com/robomaker/home?region=us-west-2#simulationJobs/sim-cwck722gh4bk)  
- [Simulation 4](https://us-west-2.console.aws.amazon.com/robomaker/home?region=us-west-2#simulationJobs/sim-4lcp9fs81k6q)  
- [Simulation 5](https://us-west-2.console.aws.amazon.com/robomaker/home?region=us-west-2#simulationJobs/sim-wyq8rsft8vhc)  
- [Simulation 6](https://us-west-2.console.aws.amazon.com/robomaker/home?region=us-west-2#simulationJobs/sim-ck8f52fg5y6k)  
- [Simulation 7](https://us-west-2.console.aws.amazon.com/robomaker/home?region=us-west-2#simulationJobs/sim-fvd48kbwfqv0)  
- [Simulation 8](https://us-west-2.console.aws.amazon.com/robomaker/home?region=us-west-2#simulationJobs/sim-2cbst0sj20bw)  

You can click on Gazebo after you open the above link to start the simulator.

### Evaluation - ReInvent Track

In [30]:
sys.path.append("./src")

num_simulation_workers = 1


envriron_vars = {
    "WORLD_NAME": "reInvent2019_track",
    "KINESIS_VIDEO_STREAM_NAME": "SilverstoneStream",
    "SAGEMAKER_SHARED_S3_BUCKET": s3_bucket,
    "SAGEMAKER_SHARED_S3_PREFIX": s3_prefix,
    "MODEL_S3_BUCKET": s3_bucket,
    "PYTHONPATHS":"wohooo this worked",
    "MODEL_S3_PREFIX": s3_prefix,
    "ALTERNATE_DRIVING_DIRECTION": "false",
    "APP_REGION": aws_region,
    "MODEL_METADATA_FILE_S3_KEY": "%s/model_metadata.json" % s3_prefix,
    "METRICS_S3_BUCKET": s3_bucket,
    "METRICS_S3_OBJECT_KEY": s3_prefix + "/evaluation_metrics.json",
    "NUMBER_OF_TRIALS": "5", # Doesnt matter
    "ROBOMAKER_SIMULATION_JOB_ACCOUNT_ID": account_id
}

simulation_application = {
    "application":simulation_app_arn,
    "launchConfig": {
         "packageName": "deepracer_simulation_environment",
         "launchFile": "evaluation.launch",
         "environmentVariables": envriron_vars
    }
}
                            
vpcConfig = {"subnets": deepracer_subnets,
             "securityGroups": deepracer_security_groups,
             "assignPublicIp": True}

responses_eval = []
for job_no in range(num_simulation_workers):
    response =  robomaker.create_simulation_job(clientRequestToken=strftime("%Y-%m-%d-%H-%M-%S", gmtime()),
                                                outputLocation={ 
                                                  "s3Bucket": s3_bucket,
                                                  "s3Prefix": s3_prefix
                                                },
                                                maxJobDurationInSeconds=job_duration_in_seconds*3,
                                                iamRole=sagemaker_role,
                                                failureBehavior="Continue",
                                                simulationApplications=[simulation_application],
                                                vpcConfig=vpcConfig)
    responses_eval.append(response)

# print("Created the following jobs:")
#for response in responses_eval:
#    print("Job ARN", response["arn"]) 
    

SyntaxError: invalid syntax (<ipython-input-30-58b3e8b6d887>, line 46)

In [ ]:
print('~~~ EVAL ~~~\n')
arn_str = list()
for response in responses_eval:
    arn_str.append(response["arn"].split('/')[-1])
print('eval_stream_name_list =' + str(arn_str))


### Clean up RoboMaker and SageMaker training job

Execute the cells below if you want to kill RoboMaker and SageMaker job.

In [ ]:
# # Cancelling robomaker job
# for job_arn in job_arns:
#     robomaker.cancel_simulation_job(job=job_arn)

# # # Stopping sagemaker training job
# sage_session.sagemaker_client.stop_training_job(TrainingJobName=estimator._current_job_name)

### Clean Up Simulation Application Resource

In [ ]:
# robomaker.delete_simulation_application(application=simulation_app_arn)

### Clean your S3 bucket (Uncomment the awscli commands if you want to do it)

In [ ]:
## Uncomment if you only want to clean the s3 bucket
# sagemaker_s3_folder = "s3://{}/{}".format(s3_bucket, s3_prefix)
# !aws s3 rm --recursive {sagemaker_s3_folder}

# robomaker_s3_folder = "s3://{}/{}".format(s3_bucket, job_name)
# !aws s3 rm --recursive {robomaker_s3_folder}

# robomaker_sim_app = "s3://{}/{}".format(s3_bucket, 'robomaker')
# !aws s3 rm --recursive {robomaker_sim_app}

# model_output = "s3://{}/{}".format(s3_bucket, s3_bucket)
# !aws s3 rm --recursive {model_output}

### Clean the docker images
Remove this only when you want to completely remove the docker or clean up the space of the sagemaker instance

In [ ]:
# !docker rmi -f $(docker images -q)